# 필요한 모듈 임포트

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# 학습, 벨리데이션 경로 및 라벨 설정

In [ ]:
dataset_dir = '/content/drive/MyDrive/KDT 3차 프로젝트/ImageFolder'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dir = os.listdir('/content/drive/MyDrive/KDT 3차 프로젝트/ImageFolder/train')
val_dir = os.listdir('/content/drive/MyDrive/KDT 3차 프로젝트/ImageFolder/validation')

In [ ]:
# GPU사용 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# 이미지 증강 기법

In [ ]:
# 이미지 증강 기법
# data_trainsforms
data_trainsforms = {
    'train': transforms.Compose([
        transforms.ToTensor()
    ]),
    'validation': transforms.Compose([
        transforms.ToTensor()
    ])
}

In [ ]:
def target_transforms(target):
    return torch.FloatTensor([target])

# 데이터셋 객체 생성

In [ ]:
# 데이터셋 객체 생성
image_datasets = {
    'train': datasets.ImageFolder('/content/drive/MyDrive/KDT 3차 프로젝트/ImageFolder/train', data_trainsforms['train']),
    'validation': datasets.ImageFolder('/content/drive/MyDrive/KDT 3차 프로젝트/ImageFolder/validation', data_trainsforms['validation']),
}

# 데이터 로더 생성


In [ ]:
dataloaders = {
    'train': DataLoader(
        image_datasets['train'],
        batch_size=16,
        shuffle=True
    )
    ,
    'validation': DataLoader(
        image_datasets['validation'],
        batch_size=16,
        shuffle=False
    )
}

In [ ]:
print(len(image_datasets['train']))
print(len(image_datasets['validation']))

In [ ]:
# train 데이터 로더 확인
imgs, labels = next(iter(dataloaders['train']))

fig, axes = plt.subplots(2, 4, figsize=(20,10))

for img, label,ax in zip(imgs, labels, axes.flatten()):
    ax.imshow(img.permute(1,2,0))
    ax.set_title(label)
    ax.axis('off')

image_datasets['train'].classes[11]

In [ ]:
class ResNetCustom(nn.Module):
    def __init__(self, num_classes=982):
        super(ResNetCustom, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.resnet.fc(x)

        return x

# 모델 생성
model = ResNetCustom().to(device)

# 모델 출력
print(model)


In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/KDT 3차 프로젝트/모델/아람/모델_다운/model_test.pth')
model.load_state_dict(checkpoint)
model.train()

In [ ]:
epochs = 10
optimizer = optim.Adam(model.resnet.fc.parameters(), lr=0.001)

for epoch in range(epochs + 1):
    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
        else:
            model.eval()
        sum_losses = 0
        sum_accs = 0
        for x_batch, y_batch in dataloaders[phase]:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            y_pred = model(x_batch).to(device)
            loss = nn.CrossEntropyLoss()(y_pred,  y_batch)
            if phase == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            sum_losses += loss.item()  # loss.item()을 사용하여 loss 값을 가져옵니다.
            y_prob = nn.Softmax(1)(y_pred)
            y_pred_index = torch.argmax(y_prob, axis=1)

            acc = (y_batch == y_pred_index).float().sum() / len(y_batch) * 100
            sum_accs += acc.item()  # acc.item()을 사용하여 acc 값을 가져옵니다.

        avg_loss = sum_losses / len(dataloaders[phase])
        avg_acc = sum_accs / len(dataloaders[phase])
        print(f'{phase:10s}: Epoch {epoch+1:4d}/{epochs} Loss: {avg_loss:.4f} Accuracy: {avg_acc:.2f}%')


In [ ]:
# 학습된 모델 파일  저장
torch.save(model.state_dict(),'/content/drive/MyDrive/KDT 3차 프로젝트/모델/아람/모델_다운/model_test2.pth')